In [ ]:
# %pip install datasets

In [ ]:
import model_one
from datasets import load_dataset
import pandas as pd

In [ ]:
model_one.cli.API_KEY = 'eyJhbGci...'

In [ ]:
dataset = load_dataset("tweet_eval", "irony")
train = pd.DataFrame(dataset['train'])
validation = pd.DataFrame(dataset['validation'])

In [ ]:
model = model_one.train_classifier(r'model-one-tweet_eval-irony.json',
                    train['text'], train['label'], validation['text'], validation['label'])
model

In [ ]:
model.status()

In [ ]:
res_validation = []
for text in dataset['validation']['text']:
    res_validation += model.classify(input=text)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

import numpy as np

precision, recall, thresholds = precision_recall_curve(dataset['validation']['label'], res_validation)
f1_scores = 2*recall*precision/(recall+precision)

threshold = thresholds[np.argmax(f1_scores)]
print('Best threshold: ', threshold)
print('Best F1-Score: ', np.max(f1_scores))

In [ ]:
res = []
for text in dataset['test']['text']:
    res += model.classify(input=text)

In [52]:
y_pred = 1*(np.array(res) > threshold)

print(classification_report(dataset['test']['label'], y_pred))

              precision    recall  f1-score   support

           0       0.87      0.61      0.72       473
           1       0.60      0.86      0.71       311

    accuracy                           0.71       784
   macro avg       0.73      0.74      0.71       784
weighted avg       0.76      0.71      0.71       784

